In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import nltk
from nltk.util import ngrams

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
train_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Train/Noakhali Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Test/Noakhali Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Validation/Noakhali Validation Translation.csv")

In [7]:
train_data.head(5)

,bangla_speech,banglish_speech,noakhali_bangla_speech,noakhali_banglish_speech,region_name,english_speech
0,কেমন আছো ?,kemon acho?,কেইক্কান আছেন ?,keikkan achen ?,Noakhali,How are you?
1,আজকে আমার মন ভালো নেই,ajke amr mon valo nei,আইজ্জা আর মন ভালা নাই,aijja ar mon vala nai,Noakhali,I'm not feeling well today
2,তুমি কি করো ?,tumi ki koro?,তুমি কিআ করর ?,tumi kiya koror ?,Noakhali,what are you doing
3,এই গরমে আমার কিছু ভালো লাগে না,ei gorome amar kichu valo lage na,এতো গরমে আর কিচ্ছু ভালা লাগে না,eto gorme ar kicchu vala lage na,Noakhali,I don't like anything this summer
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,cheleti sada ronger ekti shirt pore eshechilo,পোলাডা একটা সাদা রং এর শার্ট পড়ি আইসিলো,polada ekta sada rong er shirt pori aisilo,Noakhali,The boy came wearing a white shirt


In [8]:
train_data.tail(5)

,bangla_speech,banglish_speech,noakhali_bangla_speech,noakhali_banglish_speech,region_name,english_speech
1870,কাউকে মিথ্যা বলে খুশী করার থেকে সত্যি বলে কাঁদ...,kauke mittha bole khushi korar theke sotti bol...,কাউরে মিত্যা কই হাসানির তুন সত্যি কই কাদানি ভালা,kaure mitta ki hasanir tun sotti koi kadani vala,Noakhali,It's better to be happy for telling a lie than...
1871,"একে অপরের প্রতি যতো বেশী বিশ্বাস থাকবে, তাদের ...","eke oporer proti joto beshi bisshas thakbe, ta...",একে অফরের প্রতি যত বেশী বিশ্বাস থাইকব হেতাগো ...,eke oforer proti joto beshi bisshas thaikbo h...,Noakhali,"The more trust they have in each other, the he..."
1872,বৃষ্টি বেড়ে তাপমাত্রা কমার সুখবর দিয়েছে আবহা...,brishti bere tapmatra komar sukhobor diyeche a...,বৃষ্টি বাড়ি তাফমাত্রা কইমবার সুখবর দিছে আবহাও...,brishti bari tafmatra koimbar sukhobor diche a...,Noakhali,Meteorological office has given the good news ...
1873,পুরো বর্ষা মৌসুমে হাতে গোনা কয়েকদিন মাত্র বৃষ...,puro borsha moushume hate gona koyekdin matro ...,হুরা বর্ষার ভিত্তে আতে গোনা কয়েকদিন বৃষ্টি অই...,hura borshar vitte ate gona koyekdin brishti o...,Noakhali,After only a handful of days of rain in the en...
1874,খাওয়ার অনেক আগেই যদি ভাত রান্না শেষ হয়ে গিয়ে থ...,khaoyar onek agei jodi vat ranna shesh hoye gi...,খাইবার অনেক আগেই যদি ভাত রান্দা অই যায় তই ডাক...,khaibar onek agei jodi vat randa oi jay toi da...,Noakhali,If the rice is done cooking long before eating...


In [9]:
test_data.head(5)

,bangla_speech,banglish_speech,noakhali_bangla_speech,noakhali_banglish_speech,region_name,english_speech
0,তোমার আব্বু কেমন আছে?,tomar abbu kemon ache?,তোর আব্বা কেক্কা আছে?,tor abba kekka ache?,Noakhali,How is your father?
1,আমার বড় বোনের আজকে মন ভালো নেই,amar boro boner ajke mon valo nei,আর বড় আফার আইজ্জা মন ভালা নাই,ar boro afar aijja mon vala nai,Noakhali,My elder sister is not feeling well today
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,tumi ki amake ei kajti kore dite parbe?,তুই কি আরে এই কাম আন করি দিতা হাইরবা নি?,tui ki are ei kam an kori dita hairba ni?,Noakhali,Can you do this for me?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,ei gorome bahire ber hote iccha kore na,এই গরমেত্তে বাইর অইত ইচ্ছা করে না,ei goromette bair oito iccha kore na,Noakhali,Don't want to go out in this summer
4,তোমার সব কথাই ভুল,tomar sob kothai vul,তোর বেক কতাই ভুল,tor bek kotai vul,Noakhali,Everything you say is wrong


In [10]:
test_data.tail(5)

,bangla_speech,banglish_speech,noakhali_bangla_speech,noakhali_banglish_speech,region_name,english_speech
370,তুমি কি আমাকে এক গ্লাস পানি এনে দিতে পারবে ?,tumi ki amake ek glass pani ene dite parbe ?,তুই কি আরে এক গ্লাস হানি আনি দিতা হাইরবানি?,tui ki are ek glass hani ani dita hairbani?,Noakhali,Can you get me a glass of water?
371,আমি পারবো না পানি দিতে,ami parbo na pani dite,আই হাইত্তান্নো হানি দিতাম,ai haittanno hani ditam,Noakhali,I can't give water
372,তোমার ভাই পড়ালেখাতে অনেক মেধাবি,tomar vai poralekhate onek medhabi,তোওয়ার ভাই হড়ালেখাত অনেক মেধাবি,tooyar vai horalekhat onek medhabi,Noakhali,Your brother is very talented in studies
373,আচ্ছা বলো দেখি বাংলাদেশে কয়টি জেলা?,accha bolo dekhi bangladeshe koyti jela?,আইচ্ছা বলো সাই বাংলাদেশে কোগা জেলা?,aiccha bolo sai bangladesh e koga jela?,Noakhali,Well tell me how many districts in Bangladesh?
374,সামনের দিকে যেয়ে মেয়েটি অনেক হাসবে,samner dike jeye meyeti onek hashbe,সাম্নের মুই যাই মাইয়া ইয়া অনেক হাসিবো,samner mui jai maiya iya onek hasibo,Noakhali,The girl will smile a lot going forward


In [11]:
validation_data.head(5)

,bangla_speech,banglish_speech,noakhali_bangla_speech,noakhali_banglish_speech,region_name,english_speech
0,বাংলাদেশে ৬৪ টা জেলা,bangladeshe ৬৪ ta jela,বাংলাদেশে ৬৪ গা জেলা,bangladesh e 64 ga jela,Noakhali,64 districts in Bangladesh
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,amra sobai gotokal bahire giyechilam,আন্ডা বেজ্ঞুন কাইল্লা বাইরে গেসিলাম,anda beggun kailla baire gesilam,Noakhali,We all went out yesterday
2,তোমার কথা বলার ধরন অনেক সুন্দর,tomar kotha bolar dhoron onek sundor,তোওয়ার কথা কইবার ধরন অনেক সুন্দর,towyar kotha koibar dhoron onek sundor,Noakhali,Your way of speaking is very nice
3,বরিশালের মানুষ কেমন হয়?,barishaler manush kemon hoy?,বরিশালের মানুষ কেইচ্চা অয়?,barishal er manush keicca oy?,Noakhali,How are the people of Barisal?
4,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,Noakhali,Khulna district is very beautiful?


In [12]:
validation_data.tail(5)

,bangla_speech,banglish_speech,noakhali_bangla_speech,noakhali_banglish_speech,region_name,english_speech
245,তুমি যাও এখান থেকে,tumi jao ekhan theke,তুমি যাও ইয়ান তুন,tumi jao iyan tun,Noakhali,you go from here
246,আপনি যান এখান থেকে,apni jan ekhan theke,আন্নে যান ইয়ান তুন,anne jan iyan tun,Noakhali,You go from here
247,তুই যা এখান থেকে,tui ja ekhan theke,তুই যা ইয়ান তুন,tui ja iyan tun,Noakhali,you go from here
248,আপনারা পার্কে ঘুরে বেড়ান,apnara parke ghure beran,আন্নেরা হার্কে ঘুরি বেড়ান,annera harke ghuri beran,Noakhali,You go for a walk in the park
249,উনি চেষ্টা করেন,uni cheshta koren,হেতনে চেষ্টা করেন,hetne ceshta koren,Noakhali,He tries


In [13]:
!pip install transformers torch pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [14]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.1 MB/s eta 0:00:00


In [15]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=af035f4d5bc1a6bf9288b75c7be96e74840166e54c15618d01c9073441ccf196
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [16]:
!pip install sentencepiece

In [17]:
!pip install transformers[sentencepiece]

In [18]:
!transformers-cli cache clear

2024-06-12 17:10:57.641475: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 17:10:57.641529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 17:10:57.795552: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-12 17:10:58.003232: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 17:11:00.185340: W tensorflow/compiler/tf2

In [19]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.2 MB/s eta 0:00:00


In [20]:
!pip install accelerate -U

In [21]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-k7akl5cu
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-k7akl5cu
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=ec546f1103ad818264976755fced86574d52d0c70e23f657b40fa962b506bbd8
  Stored in directory: /tmp/pip-ephem-wheel-cache-egvp2dh4/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=1b993962d8bb58e9e67a45ed35bbf472d847a12

In [22]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [23]:
#!pip install transformers==4.10.3

In [24]:
!pip install torch transformers

In [25]:
#!pip install accelerate==0.20.3

In [26]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from normalizer import normalize
import pandas as pd
import numpy as np
import nltk
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
train_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Train/Noakhali Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Test/Noakhali Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Dataset/Vashantor_CSV_Format/Validation/Noakhali Validation Translation.csv")

In [28]:
train_data.head()

,bangla_speech,banglish_speech,noakhali_bangla_speech,noakhali_banglish_speech,region_name,english_speech
0,কেমন আছো ?,kemon acho?,কেইক্কান আছেন ?,keikkan achen ?,Noakhali,How are you?
1,আজকে আমার মন ভালো নেই,ajke amr mon valo nei,আইজ্জা আর মন ভালা নাই,aijja ar mon vala nai,Noakhali,I'm not feeling well today
2,তুমি কি করো ?,tumi ki koro?,তুমি কিআ করর ?,tumi kiya koror ?,Noakhali,what are you doing
3,এই গরমে আমার কিছু ভালো লাগে না,ei gorome amar kichu valo lage na,এতো গরমে আর কিচ্ছু ভালা লাগে না,eto gorme ar kicchu vala lage na,Noakhali,I don't like anything this summer
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,cheleti sada ronger ekti shirt pore eshechilo,পোলাডা একটা সাদা রং এর শার্ট পড়ি আইসিলো,polada ekta sada rong er shirt pori aisilo,Noakhali,The boy came wearing a white shirt


In [29]:
train_data = train_data.drop(columns=['banglish_speech ','noakhali_banglish_speech ','region_name ','english_speech'],axis=1)
train_data.head()

,bangla_speech,noakhali_bangla_speech
0,কেমন আছো ?,কেইক্কান আছেন ?
1,আজকে আমার মন ভালো নেই,আইজ্জা আর মন ভালা নাই
2,তুমি কি করো ?,তুমি কিআ করর ?
3,এই গরমে আমার কিছু ভালো লাগে না,এতো গরমে আর কিচ্ছু ভালা লাগে না
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,পোলাডা একটা সাদা রং এর শার্ট পড়ি আইসিলো


In [30]:
test_data = test_data.drop(columns=['banglish_speech ','noakhali_banglish_speech ','region_name ','english_speech'],axis=1)
test_data.head()

,bangla_speech,noakhali_bangla_speech
0,তোমার আব্বু কেমন আছে?,তোর আব্বা কেক্কা আছে?
1,আমার বড় বোনের আজকে মন ভালো নেই,আর বড় আফার আইজ্জা মন ভালা নাই
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,তুই কি আরে এই কাম আন করি দিতা হাইরবা নি?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,এই গরমেত্তে বাইর অইত ইচ্ছা করে না
4,তোমার সব কথাই ভুল,তোর বেক কতাই ভুল


In [31]:
train_data.rename(columns={'bangla_speech ': 'input_text', 'noakhali_bangla_speech '	: 'labels'}, inplace=True)
train_data.head()

,input_text,labels
0,কেমন আছো ?,কেইক্কান আছেন ?
1,আজকে আমার মন ভালো নেই,আইজ্জা আর মন ভালা নাই
2,তুমি কি করো ?,তুমি কিআ করর ?
3,এই গরমে আমার কিছু ভালো লাগে না,এতো গরমে আর কিচ্ছু ভালা লাগে না
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,পোলাডা একটা সাদা রং এর শার্ট পড়ি আইসিলো


In [32]:
# Rename the columns to match the expected format
test_data.rename(columns={'bangla_speech ': 'input_text', 'noakhali_bangla_speech '	: 'labels'}, inplace=True)
test_data.head()

,input_text,labels
0,তোমার আব্বু কেমন আছে?,তোর আব্বা কেক্কা আছে?
1,আমার বড় বোনের আজকে মন ভালো নেই,আর বড় আফার আইজ্জা মন ভালা নাই
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,তুই কি আরে এই কাম আন করি দিতা হাইরবা নি?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,এই গরমেত্তে বাইর অইত ইচ্ছা করে না
4,তোমার সব কথাই ভুল,তোর বেক কতাই ভুল


In [33]:
validation_data.head()

,bangla_speech,banglish_speech,noakhali_bangla_speech,noakhali_banglish_speech,region_name,english_speech
0,বাংলাদেশে ৬৪ টা জেলা,bangladeshe ৬৪ ta jela,বাংলাদেশে ৬৪ গা জেলা,bangladesh e 64 ga jela,Noakhali,64 districts in Bangladesh
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,amra sobai gotokal bahire giyechilam,আন্ডা বেজ্ঞুন কাইল্লা বাইরে গেসিলাম,anda beggun kailla baire gesilam,Noakhali,We all went out yesterday
2,তোমার কথা বলার ধরন অনেক সুন্দর,tomar kotha bolar dhoron onek sundor,তোওয়ার কথা কইবার ধরন অনেক সুন্দর,towyar kotha koibar dhoron onek sundor,Noakhali,Your way of speaking is very nice
3,বরিশালের মানুষ কেমন হয়?,barishaler manush kemon hoy?,বরিশালের মানুষ কেইচ্চা অয়?,barishal er manush keicca oy?,Noakhali,How are the people of Barisal?
4,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,Noakhali,Khulna district is very beautiful?


In [34]:
validation_data = validation_data.drop(columns=['banglish_speech ','noakhali_banglish_speech ','region_name ','english_speech'],axis=1)
validation_data.head()

,bangla_speech,noakhali_bangla_speech
0,বাংলাদেশে ৬৪ টা জেলা,বাংলাদেশে ৬৪ গা জেলা
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,আন্ডা বেজ্ঞুন কাইল্লা বাইরে গেসিলাম
2,তোমার কথা বলার ধরন অনেক সুন্দর,তোওয়ার কথা কইবার ধরন অনেক সুন্দর
3,বরিশালের মানুষ কেমন হয়?,বরিশালের মানুষ কেইচ্চা অয়?
4,খুলনা জেলা কি অনেক সুন্দর?,খুলনা জেলা কি অনেক সুন্দর?


In [35]:
# Rename the columns to match the expected format
validation_data.rename(columns={'bangla_speech ': 'input_text', 'noakhali_bangla_speech '	: 'labels'}, inplace=True)
validation_data.head()

,input_text,labels
0,বাংলাদেশে ৬৪ টা জেলা,বাংলাদেশে ৬৪ গা জেলা
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,আন্ডা বেজ্ঞুন কাইল্লা বাইরে গেসিলাম
2,তোমার কথা বলার ধরন অনেক সুন্দর,তোওয়ার কথা কইবার ধরন অনেক সুন্দর
3,বরিশালের মানুষ কেমন হয়?,বরিশালের মানুষ কেইচ্চা অয়?
4,খুলনা জেলা কি অনেক সুন্দর?,খুলনা জেলা কি অনেক সুন্দর?


In [36]:
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer
#https://huggingface.co/docs/transformers/model_doc/mt5
model_name = "google/mt5-small" # The variations it has -> mt5-small: 6, mt5-base: 12,mt5-large: 24, mt5-xl: 24, mt5-xxl: 24
model = MT5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [37]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [38]:
import torch
from normalizer import normalize
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os

In [39]:
""""model_name = "csebuetnlp/banglat5"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

SyntaxError: incomplete input (<ipython-input-39-2a5ca7da0b93>, line 1)

In [40]:
from torch.utils.data import Dataset, DataLoader

In [41]:
class Seq2SeqDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        """
        Args:
            data: A DataFrame containing 'input_text' and 'labels' columns.
            tokenizer: A Hugging Face tokenizer.
            max_length: Maximum sequence length.
        """
        self.input_text = data['input_text'].apply(normalize).tolist()
        self.labels = data['labels'].apply(normalize).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, idx):
        input_text = self.input_text[idx]
        label_text = self.labels[idx]

        # Tokenize the input text
        input_encodings = self.tokenizer(
            input_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Tokenize the label text to get its 'input_ids' and 'attention_mask'
        label_encodings = self.tokenizer(
            label_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encodings['input_ids'].squeeze(),
            'attention_mask': input_encodings['attention_mask'].squeeze(),
            'labels': label_encodings['input_ids'].squeeze(),
        }


In [42]:
# Create train , test and validation datasets
train_dataset = Seq2SeqDataset(train_data, tokenizer)
test_dataset = Seq2SeqDataset(test_data, tokenizer)
validation_dataset = Seq2SeqDataset(validation_data, tokenizer)

# Create train , test and validation dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)  #batch_size=32
test_dataloader = DataLoader(test_dataset, batch_size=16) #batch_size=32
validation_dataloader = DataLoader(validation_dataset, batch_size=16) #batch_size=32


In [43]:
# Move the model to the device (CPU or GPU)
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [ ]:
#pip install transformers==4.30


In [44]:
from transformers import TrainingArguments
from transformers import AdamW

from torch.optim import AdamW

# Create a custom optimizer using torch.optim.AdamW
custom_optimizer = AdamW(
    model.parameters(),
    lr=1e-3,  # Learning rate
    eps=1e-8,  # Epsilon value to prevent division by zero
    weight_decay=0.01,  # Weight decay (L2 regularization)
)

#if you have 1,000 training examples and a batch size of 100, you would have 10 iterations in each epoch (1,000 / 100 = 10)
'''
This parameter determines how many small batches are accumulated before performing a weight update.
In your code, it's set to 8, which means you'll accumulate gradients over 8 small batches before performing a weight update.
This effectively simulates a larger batch size without requiring more GPU memory.
So, you are updating the model's weights less frequently compared to the number of actual batches processed.
'''



'''
Learning rate determines how quickly the model learns from the data.
The learning rate scheduler type is set to "cosine_with_restarts," which is a type of learning rate schedule.
 Warmup steps are the number of initial training steps with a smaller learning rate, and weight decay introduces L2 regularization to the optimizer.
'''



'''
Number of Iterations per Epoch = Number of Training Samples / Batch Size
Total Iterations = Number of Iterations per Epoch * Number of Epochs
'''
# Define the TrainingArguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Movies',
    num_train_epochs=50,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="epoch",
    save_strategy="no",  # Align save strategy with evaluation strategy
    ##save_total_limit=1,
    #save_steps=700,
    learning_rate=1e-3,
    do_train=True,
    do_eval=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    #load_best_model_at_end=True,
    lr_scheduler_type="cosine_with_restarts",
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/Movies',
    logging_steps=10,  # Log every 10 steps,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [45]:
from transformers import DataCollatorForSeq2Seq

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,  # Your Hugging Face tokenizer
    model=model,
    padding=True,
    max_length=128,
    label_pad_token_id=tokenizer.pad_token_id,
)


In [46]:

# Define the Trainer with the custom optimizer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=(custom_optimizer, None),  # Pass the custom optimizer here
)


# **Training start here**

In [ ]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
0,1.192900,0.554774


In [ ]:
'''from transformers import AutoModelForSeq2SeqLM

# Save the model
model.save_pretrained("/content/drive/MyDrive/Movies/Noakhali_model.pt")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Movies/Noakhali_tokenizer.json")


In [ ]:
!pip install datasets

In [ ]:
!pip install python-Levenshtein

In [ ]:
!pip install jiwer

In [ ]:
# Move the model to the device (CPU or GPU)
model.to(device)

In [ ]:
!pip install rouge-score
#https://github.com/google-research/google-research/tree/master/rouge
#https://huggingface.co/spaces/evaluate-metric/rouge [Different types of ROUGE scores]

In [ ]:
!pip install evaluate

# **Loading evaluation metrics**

In [ ]:
import torch
import Levenshtein
from evaluate import load
# Define the move_to_device function
def move_to_device(batch, device):
    if isinstance(batch, torch.Tensor):
        return batch.to(device)
    elif isinstance(batch, list):
        return [move_to_device(item, device) for item in batch]
    elif isinstance(batch, dict):
        return {key: move_to_device(value, device) for key, value in batch.items()}
    else:
        return batch  # If it's not a tensor, list, or dict, leave it as is

# Load the evaluation metric for Character Error Rate (CER) and Word Error Rate (WER) and Exact Match(em)
cer_metric = load("cer")
wer_metric = load("wer")
meteor = load('meteor')
exact_match_metric = load("exact_match")

# Load BLEU and ROUGE metrics
bleu_metric = load("bleu")
rouge_metric = load('rouge')

# Initialize lists to store generated translations and references
generated_translations = []
references = []

# Generate translations for the test dataset
for batch in test_dataloader:
    # Move the batch to CUDA
    batch = move_to_device(batch, 'cuda')

    input_text = batch['input_ids']  # Access the input_text using the correct key
    labels = batch['labels']  # Access the labels using the correct key

    # Generate translations
    translation_ids = model.generate(input_text, max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)

    # Move the translation_ids to CPU to decode
    translation_ids = translation_ids.to('cpu')

    generated_translation = tokenizer.batch_decode(translation_ids, skip_special_tokens=True)

    generated_translations.extend(generated_translation)
    references.extend(tokenizer.batch_decode(labels, skip_special_tokens=True))  # Decoding the label IDs

# Make sure to move generated_translations back to CPU for evaluation if necessary
generated_translations = [translation if not isinstance(translation, str) else translation for translation in generated_translations]



In [ ]:
# Calculate Character Error Rate (CER) and Word Error Rate (WER)
results_CER = cer_metric.compute(predictions=generated_translations, references=references)
results_WER = wer_metric.compute(predictions=generated_translations, references=references)

# Calculate Exact Match (EM) and METEOR(M)
results_em = exact_match_metric.compute(predictions=generated_translations, references=references)
results_met = meteor.compute(predictions=generated_translations, references=references)

# Calculate Bilingual Evaluation Understudy (BLEU) and Recall-Oriented Understudy for Gisting Evaluation (ROUGE)
results_bleu = bleu_metric.compute(predictions=generated_translations, references=references)
results_rouge = rouge_metric.compute(predictions=generated_translations, references=references)


# Calculate Levenshtein Distance
levenshtein_distances = [Levenshtein.distance(generated, reference) for generated, reference in zip(generated_translations, references)]


# **Printing every evaluation metrics**

In [ ]:
print(results_CER)
print(results_WER)
print(results_em)
print(results_met)
print(results_bleu)
print(results_rouge)
print(levenshtein_distances)

In [ ]:
'''total_correct = 0
total_samples = len(references)

for generated, reference in zip(generated_translations, references):
    levenshtein_distance = Levenshtein.distance(generated, reference)
    max_length = max(len(generated), len(reference))
    accuracy = 1 - (levenshtein_distance / max_length)
    if accuracy >= 0.8:  # Adjust the threshold as needed
        total_correct += 1

accuracy = total_correct / total_samples
print("Accuracy:", accuracy)
